# 200 : Rethink

Approaches from phase 1 does not work at all for the new data. We need to rethink our approach.

In [19]:
import numpy as np
import pandas as pd
import logging

from prophet import Prophet

## Data Collection

In [2]:
# load the source training data
df_source = pd.read_csv('../../data/input/df_train.csv')

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

(8763, 49)


,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Madrid_clouds_all,Seville_wind_speed,Barcelona_rain_1h,Seville_pressure,Seville_rain_1h,Bilbao_snow_3h,Barcelona_pressure,Seville_rain_3h,Madrid_rain_1h,Barcelona_rain_3h,Valencia_snow_3h,Madrid_weather_id,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_temp,Bilbao_weather_id,Seville_temp,Valencia_humidity,Valencia_temp_min,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.0,0.0,223.333333,6.333333,42.666667,0.0,3.333333,0.0,sp25,0.0,0.0,1036.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.000000,800.0,1002.666667,274.254667,971.333333,269.888000,269.888000,800.0,274.254667,75.666667,269.888000,281.013000,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.0,0.0,221.000000,4.000000,139.000000,0.0,3.333333,0.0,sp25,0.0,0.0,1037.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.666667,800.0,1004.333333,274.945000,972.666667,271.728333,271.728333,800.0,274.945000,71.000000,271.728333,280.561667,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.0,0.0,214.333333,2.000000,326.000000,0.0,2.666667,0.0,sp25,0.0,0.0,1038.000000,0.0,0.0,0.0,0.0,800.0,800.0,1036.000000,800.0,1005.333333,278.792000,974.000000,278.008667,278.008667,800.0,278.792000,65.666667,278.008667,281.583667,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667


In [3]:
# load the competition data
df_competition = pd.read_csv('../../data/input/df_test.csv')

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

(8763, 49)


,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Madrid_clouds_all,Seville_wind_speed,Barcelona_rain_1h,Seville_pressure,Seville_rain_1h,Bilbao_snow_3h,Barcelona_pressure,Seville_rain_3h,Madrid_rain_1h,Barcelona_rain_3h,Valencia_snow_3h,Madrid_weather_id,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_temp,Bilbao_weather_id,Seville_temp,Valencia_humidity,Valencia_temp_min,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.0,0.0,223.333333,6.333333,42.666667,0.0,3.333333,0.0,sp25,0.0,0.0,1036.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.000000,800.0,1002.666667,274.254667,971.333333,269.888000,269.888000,800.0,274.254667,75.666667,269.888000,281.013000,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.0,0.0,221.000000,4.000000,139.000000,0.0,3.333333,0.0,sp25,0.0,0.0,1037.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.666667,800.0,1004.333333,274.945000,972.666667,271.728333,271.728333,800.0,274.945000,71.000000,271.728333,280.561667,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.0,0.0,214.333333,2.000000,326.000000,0.0,2.666667,0.0,sp25,0.0,0.0,1038.000000,0.0,0.0,0.0,0.0,800.0,800.0,1036.000000,800.0,1005.333333,278.792000,974.000000,278.008667,278.008667,800.0,278.792000,65.666667,278.008667,281.583667,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667


## Data Cleaning

In [4]:
def clean_unused_columns(data:pd.DataFrame) -> pd.DataFrame:
    """
    Remove unused columns from the dataset
    """
    df_clean = data.copy()
    df_clean.drop(columns=['Unnamed: 0'], inplace=True)

    return df_clean

In [5]:
def clean_datatypes(data:pd.DataFrame) -> pd.DataFrame:
    """
    Set the columns to the correct datatypes
    """
    df_clean = data.copy()
    df_clean['time'] = pd.to_datetime(df_clean['time'])

    return df_clean

In [6]:
def clean_valencia_pressure(data:pd.DataFrame) -> pd.DataFrame:
    """
    Replace the nulls in Valencia_pressure with the mode
    """
    df_clean = data.copy()

    # forward fill empty values in Valencia_pressure
    df_clean['Valencia_pressure'] = df_clean['Valencia_pressure'] \
        .fillna(method='ffill')

    return df_clean

In [7]:
def clean_valencia_wind(data:pd.DataFrame) -> pd.DataFrame:
    """
    Clean Valencia wind degrees by striping string & convert to numerical
    """
    df_clean = data.copy()

    df_clean['Valencia_wind_deg'] = df_clean['Valencia_wind_deg'].str.extract('(\\d+)')
    df_clean['Valencia_wind_deg'] = pd.to_numeric(df_clean['Valencia_wind_deg'])

    return df_clean

In [8]:
def clean_seville_pressure(data:pd.DataFrame) -> pd.DataFrame:
    """
    Remove non-numeric values
    """
    df_clean = data.copy()

    df_clean['Seville_pressure'] = df_clean['Seville_pressure'] \
        .str.extract('(\\d+)') \
        .astype(float)

    return df_clean


## Feature Engineering

In [9]:
df_temp = clean_datatypes(df_source)

In [10]:
def add_date_features(data:pd.DataFrame) -> pd.DataFrame:
    """
    Split the date into its separate parts (years, mont, etc.)
    """
    df_clean = data.copy()

    df_clean['Year'] = df_clean['time'].dt.year
    df_clean['Month'] = df_clean['time'].dt.month
    df_clean['Day'] = df_clean['time'].dt.day
    df_clean['Hour'] = df_clean['time'].dt.hour
    df_clean['Day_of_week'] = df_clean['time'].dt.dayofweek

    return df_clean

In [11]:
def add_season_feature(data:pd.DataFrame) -> pd.DataFrame:
    """
    Add a season feature based on the month.
    """
    df_clean = data.copy()

    # coding for the seasons
    season = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
    
    # add a season column based on the mapping
    df_clean['Season'] = df_clean.time.dt.month.map(season)

    return df_clean

In [12]:
def add_day_features(data:pd.DataFrame) -> pd.DataFrame:
    """
    Add a feature to indicate if it is a weekday
    """
    df_clean = data.copy()
    day_type = []

    day_mapping = {
        0: 'Weekday', 1: 'Weekday', 2: 'Weekday', 3: 'Weekday',
        4: 'Weekday', 5: 'Weekend', 6: 'Weekend'}

    # add a day type based on the mapping
    df_clean['day_type'] = df_clean.time.dt.day_of_week.map(day_mapping)

    return df_clean

In [13]:
def add_lag_feature(data:pd.DataFrame, column:str, lags:int) -> pd.DataFrame:
    """
    Add lag features to the dataset
    """
    df_clean = data.copy()

    for lag in range(1, lags + 1):
        df_clean[f'{column}_lag_{lag}'] = df_clean[column].shift(lag)

    return df_clean

In [14]:
def add_lag_features(data:pd.DataFrame, lags:int) -> pd.DataFrame:
    """Add lag features to the dataset"""

    df_features = data.copy()
    cities = ['Madrid', 'Valencia', 'Seville', 'Bilbao', 'Barcelona']

    measures = [
        'wind_speed',
        'wind_deg',
        'rain_1h',
        'rain_3h',
        'humidity',
        'clouds_all',
        'pressure',
        'snow_3h',
        'weather_id',
        'temp_max',
        'temp_min',
        'temp'
    ]

    for city in cities:
        for feature in measures:
            if f'{city}_{feature}' in df_features.columns:
                df_features = add_lag_feature(df_features, f'{city}_{feature}', lags)
            else:
                logging.warn(f'{city}_{feature} not in dataset')

    return df_features

In [15]:
def add_rolling_average_features(data:pd.DataFrame, window:int) -> pd.DataFrame:
    """
    Add rolling average features to the dataset
    """
    df_features = data.copy()
    cities = ['Madrid', 'Valencia', 'Seville', 'Bilbao', 'Barcelona']

    measures = [
        'wind_speed',
        'wind_deg',
        'rain_1h',
        'rain_3h',
        'humidity',
        'clouds_all',
        'pressure',
        'snow_3h',
        'weather_id',
        'temp_max',
        'temp_min',
        'temp'
    ]

    for city in cities:
        for feature in measures:
            if f'{city}_{feature}' in df_features.columns:
                df_features[f'{city}_{feature}_rolling_avg_{window}'] = df_features[f'{city}_{feature}'].rolling(window=window).mean()
            else:
                logging.warn(f'{city}_{feature} not in dataset')

    return df_features


In [17]:
def feature_engineering(data:pd.DataFrame) -> pd.DataFrame:
    """
    Clean the dataset and add the features
    """
    df_clean = data.copy()

    # define categorical columns
    categorical_columns = [
        'Month', 'Day', 'Hour', 
        'Day_of_week', 'Season', 'Day_type'
    ]

    # perform cleaning and feature engineering
    df_clean = clean_datatypes(df_clean)
    df_clean = clean_unused_columns(df_clean)

    df_clean = clean_valencia_pressure(df_clean)
    df_clean = clean_valencia_wind(df_clean)
    df_clean = clean_seville_pressure(df_clean)

    df_clean = add_date_features(df_clean)
    df_clean = add_season_feature(df_clean)
    df_clean = add_day_features(df_clean)

    # # add rolling average features
    # df_clean = add_rolling_average_features(df_clean, 5)
    # df_clean = add_rolling_average_features(df_clean, 10)
    # df_clean = add_rolling_average_features(df_clean, 30)
    # df_clean = add_rolling_average_features(df_clean, 60)
    # df_clean = add_rolling_average_features(df_clean, 90)
    # df_clean = add_rolling_average_features(df_clean, 365)


    # # add lag features
    # df_clean = add_lag_features(df_clean, 10)
    
    # replace empty lag values with 0
    df_clean.fillna(0, inplace=True)

    return pd.get_dummies(
        df_clean, 
        drop_first=True)
    

    # return pd.get_dummies(
    #     df_clean, 
    #     columns=categorical_columns,
    #     drop_first=True)

# clean the dataset and add new features
df_features = feature_engineering(df_source)

with pd.option_context('display.max_columns', None):
    display(df_features.head(3))

,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Madrid_clouds_all,Seville_wind_speed,Barcelona_rain_1h,Seville_pressure,Seville_rain_1h,Bilbao_snow_3h,Barcelona_pressure,Seville_rain_3h,Madrid_rain_1h,Barcelona_rain_3h,Valencia_snow_3h,Madrid_weather_id,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_temp,Bilbao_weather_id,Seville_temp,Valencia_humidity,Valencia_temp_min,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h,Year,Month,Day,Hour,Day_of_week,Season_Spring,Season_Summer,Season_Winter,day_type_Weekend
0,2015-01-01 03:00:00,0.666667,5,0.0,0.666667,74.333333,64.000000,0.0,1.0,0.0,223.333333,6.333333,42.666667,0.0,3.333333,0.0,25.0,0.0,0.0,1036.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.000000,800.0,1002.666667,274.254667,971.333333,269.888000,269.888000,800.0,274.254667,75.666667,269.888000,281.013000,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667,2015,1,1,3,3,0,0,1,0
1,2015-01-01 06:00:00,0.333333,10,0.0,1.666667,78.333333,64.666667,0.0,1.0,0.0,221.000000,4.000000,139.000000,0.0,3.333333,0.0,25.0,0.0,0.0,1037.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.666667,800.0,1004.333333,274.945000,972.666667,271.728333,271.728333,800.0,274.945000,71.000000,271.728333,280.561667,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667,2015,1,1,6,3,0,0,1,0
2,2015-01-01 09:00:00,1.000000,9,0.0,1.000000,71.333333,64.333333,0.0,1.0,0.0,214.333333,2.000000,326.000000,0.0,2.666667,0.0,25.0,0.0,0.0,1038.000000,0.0,0.0,0.0,0.0,800.0,800.0,1036.000000,800.0,1005.333333,278.792000,974.000000,278.008667,278.008667,800.0,278.792000,65.666667,278.008667,281.583667,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667,2015,1,1,9,3,0,0,1,0


## Model Training

### Prophet

In [20]:
# get the data for training
df_train = feature_engineering(df_source)

# rename the columns
df_train.rename(columns={'load_shortfall_3h': 'y', 'Year': 'ds'}, inplace=True)

# Create the Prophet model
prophet_model = Prophet()

# Fit the model to the training data
prophet_model.fit(df_train)

10:09:15 - cmdstanpy - INFO - Chain [1] start processing
10:09:16 - cmdstanpy - INFO - Chain [1] done processing
10:09:16 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
10:09:16 - cmdstanpy - INFO - Chain [1] start processing
10:09:16 - cmdstanpy - INFO - Chain [1] done processing
10:09:16 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/opt/homebrew/anaconda3/envs/paula/lib/python3.8/site-packages/prophet/stan_model/prophet_model.bin random seed=34468 data file=/var/folders/6p/mfdk5t710cq33z7twx0hrc400000gn/T/tmp3aoub80a/h68d403r.json init=/var/folders/6p/mfdk5t710cq33z7twx0hrc400000gn/T/tmp3aoub80a/fdwo3pq2.json output file=/var/folders/6p/mfdk5t710cq33z7twx0hrc400000gn/T/tmp3aoub80a/prophet_modelgz65a8rh/prophet_model-20230324100916.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[79312]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <3516FF20-F08A-375A-8450-D7CA036C88E7> /opt/homebrew/anaconda3/envs/paula/lib/python3.8/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/6p/mfdk5t710cq33z7twx0hrc400000gn/T/pip-install-ttp8wbng/prophet_d78b9efab2aa48708ab3d6e85feb991d/build/lib.macosx-11.1-arm64-cpython-38/prophet/stan_model/cmdstan-2.26.1/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no